In [1]:
from sklearn import tree
import pandas as pd
from sklearn.cross_validation import KFold
import sklearn as sk
import numpy as np
import datetime as dt

train_file = 'occupancy_data/datatraining.txt'
test_file = 'occupancy_data/datatest.txt'

df = pd.read_table(train_file, delimiter = ',')
test_df = pd.read_csv(test_file, delimiter = ',')

# df['date'] = df['date'].map(pd.to_datetime)
# start = df.iloc[0,0]
# df['date'] = df['date'].apply(lambda x: x.microsecond)

/Users/shivakanththudi/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
test_df.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
140,2015-02-02 14:19:00,23.7000,26.272,585.200000,749.200000,0.004764,1
141,2015-02-02 14:19:59,23.7180,26.290,578.400000,760.400000,0.004773,1
142,2015-02-02 14:21:00,23.7300,26.230,572.666667,769.666667,0.004765,1
143,2015-02-02 14:22:00,23.7225,26.125,493.750000,774.750000,0.004744,1
144,2015-02-02 14:23:00,23.7540,26.200,488.600000,779.000000,0.004767,1


In [3]:
X_train = df.iloc[:,1:6]
X_test = test_df.iloc[:,1:6]
X_train.head()

,Temperature,Humidity,Light,CO2,HumidityRatio
1,23.18,27.2720,426.0,721.25,0.004793
2,23.15,27.2675,429.5,714.00,0.004783
3,23.15,27.2450,426.0,713.50,0.004779
4,23.15,27.2000,426.0,708.25,0.004772
5,23.10,27.2000,426.0,704.50,0.004757


In [4]:
X_test.head()

,Temperature,Humidity,Light,CO2,HumidityRatio
140,23.7000,26.272,585.200000,749.200000,0.004764
141,23.7180,26.290,578.400000,760.400000,0.004773
142,23.7300,26.230,572.666667,769.666667,0.004765
143,23.7225,26.125,493.750000,774.750000,0.004744
144,23.7540,26.200,488.600000,779.000000,0.004767


In [5]:
Y_train = df.iloc[:,6:7]
Y_test = test_df.iloc[:,6:7]

In [6]:
Y_train.head()

,Occupancy
1,1
2,1
3,1
4,1
5,1


In [7]:
Y_test.head()

,Occupancy
140,1
141,1
142,1
143,1
144,1


In [8]:
depth_list = range(1,20)
MSE_list = []
kf = KFold(len(Y_train), n_folds=10, shuffle = True, random_state  = 0)

In [9]:
for depth in depth_list:
    
    MSE_depth_CV =[]
    
    for train_index, test_index in kf:
        X_train_CV, X_test_CV = X_train.iloc[train_index,:], X_train.iloc[test_index,:]
        Y_train_CV, Y_test_CV = Y_train.iloc[train_index,:], Y_train.iloc[test_index,:]
        clf = tree.DecisionTreeClassifier(max_depth = depth) 
        clf.fit(X_train_CV, Y_train_CV) 
        predicted_y = clf.predict(X_test_CV)
        MSE = sk.metrics.accuracy_score(Y_test_CV, predicted_y)
        MSE_depth_CV.append(MSE)
    
    print depth, "k-fold CV MSE is", 1 - np.mean(MSE_depth_CV)

1 k-fold CV MSE is 0.0124030388448
2 k-fold CV MSE is 0.0116663903167
3 k-fold CV MSE is 0.0115432387212
4 k-fold CV MSE is 0.0105604377384
5 k-fold CV MSE is 0.00675374202982
6 k-fold CV MSE is 0.00687659215267
7 k-fold CV MSE is 0.00712229239837
8 k-fold CV MSE is 0.00699929153917
9 k-fold CV MSE is 0.00724499178487
10 k-fold CV MSE is 0.00761369288977
11 k-fold CV MSE is 0.00736814338041
12 k-fold CV MSE is 0.00810479190847
13 k-fold CV MSE is 0.00798209252197
14 k-fold CV MSE is 0.00785939313547
15 k-fold CV MSE is 0.00785939313547
16 k-fold CV MSE is 0.00835079362687
17 k-fold CV MSE is 0.00798239399466
18 k-fold CV MSE is 0.00822779276767
19 k-fold CV MSE is 0.00835064289052


In [10]:
X_train_CV.head()

,Temperature,Humidity,Light,CO2,HumidityRatio
2,23.15,27.2675,429.5,714.00,0.004783
3,23.15,27.2450,426.0,713.50,0.004779
4,23.15,27.2000,426.0,708.25,0.004772
5,23.10,27.2000,426.0,704.50,0.004757
6,23.10,27.2000,419.0,701.00,0.004757


In [11]:
from sklearn.ensemble import BaggingClassifier

In [12]:
n_estimators = range(1,10)

In [13]:
    for e in n_estimators:
        
        classifier = BaggingClassifier(n_estimators=e)
        classifier.fit(X_train, Y_train.values.ravel())
        predicted_y = classifier.predict(X_test)
        # print("Accuracy on training set: {:.3f}".format(classifier.score(X_train, Y_train))) 
        print("Accuracy on testing set: {:.3f}".format(classifier.score(X_test, Y_test))) 

Accuracy on testing set: 0.947
Accuracy on testing set: 0.817
Accuracy on testing set: 0.940
Accuracy on testing set: 0.913
Accuracy on testing set: 0.939
Accuracy on testing set: 0.914
Accuracy on testing set: 0.906
Accuracy on testing set: 0.899
Accuracy on testing set: 0.942


In [15]:
print("scikit-learn version: {}".format(sk.__version__))

scikit-learn version: 0.18.1


In [17]:
for e in n_estimators:
    from sklearn.ensemble import RandomForestClassifier 
    classifier = RandomForestClassifier(n_estimators=e)
    classifier.fit(X_train, Y_train.values.ravel()) 
    print("Accuracy on testing set: {:.3f}".format(classifier.score(X_test, Y_test)))


Accuracy on testing set: 0.932
Accuracy on testing set: 0.815
Accuracy on testing set: 0.914
Accuracy on testing set: 0.899
Accuracy on testing set: 0.934
Accuracy on testing set: 0.911
Accuracy on testing set: 0.934
Accuracy on testing set: 0.916
Accuracy on testing set: 0.937


In [18]:
from sklearn.ensemble import AdaBoostClassifier 
rate = [0.025,0.1, 1.0]    # Learning rate; default is 1.0

for r in rate:
    classifier = AdaBoostClassifier(learning_rate=r) 
    classifier.fit(X_train, Y_train.values.ravel()) 
    print("Accuracy on testing set: {:.3f}".format(classifier.score(X_test, Y_test)))


Accuracy on testing set: 0.979
Accuracy on testing set: 0.978
Accuracy on testing set: 0.971
